# Data Conversion of 19plus V2 .hex data
This python notebook shows how to convert a .hex file into pandas dataframe with scientific values.

Please contact [SBS customer support](https://www.seabird.com/support) for help or to request additional features.

## Init

The initialization code section below is used to import required libraries.

In [ ]:
# Native imports
import os

# Third-party imports
import numpy as np
import pandas as pd

# Sea-Bird imports
import seabirdscientific.conversion as conv
import seabirdscientific.instrument_data as id
import seabirdscientific.visualization as viz
import example_data.example_coefficients as ec

## [Data Conversion](#proc-list)

This section shows how to convert raw data contained in a .hex file into engineering units for the instruments that follow:  
- 19plus V2

In [ ]:
hex_file = os.path.join("example_data", "19plus_V2", "19plus_V2.hex")

# Convert raw hexadecimal string to raw frequencies
raw_data = id.read_hex_file(
    filepath=hex_file,
    instrument_type=id.InstrumentType.SBE19Plus,
    enabled_sensors=[
        id.Sensors.Temperature,
        id.Sensors.Conductivity,
        id.Sensors.Pressure,
        id.Sensors.ExtVolt0,
        id.Sensors.ExtVolt1,
        id.Sensors.ExtVolt2,
        id.Sensors.ExtVolt3,
    ],
)

In [ ]:
# Convert raw frequencies to scientific values

temperature = conv.convert_temperature(
    temperature_counts_in=raw_data["temperature"].values,
    coefs=ec.temperature_coefs_sn6130,
    standard='IPTS68',
    units='C',
    use_mv_r=True,
)

pressure = conv.convert_pressure(
    pressure_count=raw_data["pressure"].values,
    compensation_voltage=raw_data["temperature compensation"],
    coefs=ec.pressure_coefs_sn6130,
    units='psia',
)

conductivity = conv.convert_conductivity(
    conductivity_count=raw_data["conductivity"].values,
    temperature=temperature,
    pressure=pressure,
    coefs=ec.conductivity_coefs_sn6130
)

# Flag to be used in data processing
flag = np.zeros(len(temperature))

data = pd.DataFrame({
    "temp": temperature,
    "cond":conductivity,
    "press":pressure,
    "flag":flag
})
data

## [Data Plotting](#proc-list)

In [ ]:
config = viz.ChartConfig(
    title="19plus V2 Data Conversion",
    x_names=["temp", "cond"],
    y_names=["press"],
    z_names=[],
    chart_type="overlay",
    plot_loop_edit_flags=False,
    lift_pen_over_bad_data=True,
)

chart_data = viz.ChartData(data, config)
fig = viz.plot_xy_chart(chart_data, config)

# plotly customizations
fig["layout"]["yaxis"]["autorange"] = "reversed"
fig.data[0].name = "Temperature"
fig.data[1].name = "Conductivity"
fig["layout"]["yaxis"]["title"] = "Pressure [PSIA]"
fig["layout"]["xaxis"]["title"] = "Temperature [ITS-90 degrees C]"
fig["layout"]["xaxis2"]["title"] = "Conductivity [S/m]"

fig.update_layout(height=800)
fig.show()